In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import joblib

In [37]:
# Load Dataset
df = pd.read_csv('customer_support_tickets.csv')
df

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8464,8465,David Todd,adam28@example.net,22,Female,LG OLED,2021-12-08,Product inquiry,Installation support,My {product_purchased} is making strange noise...,Open,NaN,Low,Phone,NaN,NaN,NaN
8465,8466,Lori Davis,russell68@example.com,27,Female,Bose SoundLink Speaker,2020-02-22,Technical issue,Refund request,I'm having an issue with the {product_purchase...,Open,NaN,Critical,Email,NaN,NaN,NaN
8466,8467,Michelle Kelley,ashley83@example.org,57,Female,GoPro Action Camera,2021-08-17,Technical issue,Account access,I'm having an issue with the {product_purchase...,Closed,Eight account century nature kitchen.,High,Social media,2023-06-01 09:44:22,2023-06-01 04:31:22,3.0
8467,8468,Steven Rodriguez,fpowell@example.org,54,Male,PlayStation,2021-10-16,Product inquiry,Payment issue,I'm having an issue with the {product_purchase...,Closed,We seat culture plan.,Medium,Email,2023-06-01 18:28:24,2023-06-01 05:32:24,3.0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

In [40]:
df.describe()

,Ticket ID,Customer Age,Customer Satisfaction Rating
count,8469.000000,8469.000000,2769.000000
mean,4235.000000,44.026804,2.991333
std,2444.934048,15.296112,1.407016
min,1.000000,18.000000,1.000000
25%,2118.000000,31.000000,2.000000
50%,4235.000000,44.000000,3.000000
75%,6352.000000,57.000000,4.000000
max,8469.000000,70.000000,5.000000


In [42]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [49]:
common = df['Ticket Subject'].value_counts().head(10)
common

Ticket Subject
Refund request           576
Software bug             574
Product compatibility    567
Delivery problem         561
Hardware issue           547
Battery life             542
Network problem          539
Installation support     530
Product setup            529
Payment issue            526
Name: count, dtype: int64

In [23]:
# Target Column
target = "Customer Satisfaction Rating"

# Drop rows where target is missing
df = df.dropna(subset=[target])
print("Rows after removing missing target:", df.shape)

Rows after removing missing target: (2769, 17)


In [24]:
# Split Features and Target
X = df.drop(columns=[target])
y = df[target]

In [22]:
# Remove unwanted identifier/time columns if present
drop_cols = [c for c in X.columns if any(word in c.lower() 
              for word in ["id", "ticket", "date", "time", "email", "name"])]
X = X.drop(columns=drop_cols, errors='ignore')
print("Features used:", X.columns.tolist())

Features used: ['Customer Age', 'Customer Gender', 'Product Purchased', 'Resolution']


In [25]:
# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

In [26]:
# Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="MISSING")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features),
])

In [27]:
# Model
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced'
)

pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                           ("model", model)])

In [28]:
# Split train & test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "| Test:", X_test.shape)

Train: (2215, 16) | Test: (554, 16)


In [29]:
# Train the model
pipeline.fit(X_train, y_train)
print("Model Training Completed!")

Model Training Completed!


In [30]:
# Evaluation
y_pred = pipeline.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.19855595667870035

Classification Report:
               precision    recall  f1-score   support

         1.0       0.26      0.21      0.23       111
         2.0       0.17      0.15      0.16       110
         3.0       0.19      0.38      0.26       116
         4.0       0.21      0.17      0.19       108
         5.0       0.17      0.07      0.10       109

    accuracy                           0.20       554
   macro avg       0.20      0.20      0.19       554
weighted avg       0.20      0.20      0.19       554


Confusion Matrix:
 [[23 16 53  9 10]
 [15 17 47 23  8]
 [20 27 44 15 10]
 [16 25 37 18 12]
 [15 18 47 21  8]]


In [31]:
# ROC AUC (only if binary classification)
if len(np.unique(y_test)) == 2:
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_prob)
    print("\nROC AUC:", auc)

In [32]:
# Feature Importance
# Extract transformed feature names
onehot = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
cat_feature_names = onehot.get_feature_names_out(categorical_features)

feature_names = list(numeric_features) + list(cat_feature_names)
importances = pipeline.named_steps['model'].feature_importances_

feat_imp = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False)

print("\nTop 10 Important Features:\n")
print(feat_imp.head(10))


Top 10 Important Features:

                          feature  importance
0                       Ticket ID    0.019073
1                    Customer Age    0.016627
4380       Customer Gender_Female    0.004358
4381         Customer Gender_Male    0.004294
9504          Ticket Priority_Low    0.004195
9505       Ticket Priority_Medium    0.004108
9506          Ticket Channel_Chat    0.004035
9502     Ticket Priority_Critical    0.004035
9508         Ticket Channel_Phone    0.004030
9509  Ticket Channel_Social media    0.004012


In [33]:
# Save Model
joblib.dump(pipeline, "C:/Users/NewAdmin/OneDrive/Desktop/DS/Customer Satisfaction Prediction/customer_satisfaction_model.joblib")
print("\nModel Saved as:C:/Users/NewAdmin/OneDrive/Desktop/DS/Customer Satisfaction Prediction/customer_satisfaction_model.joblib")


Model Saved as:C:/Users/NewAdmin/OneDrive/Desktop/DS/Customer Satisfaction Prediction/customer_satisfaction_model.joblib
